# Unsupervised Semantic Analysis tutorial

The **SemanticDetector** class is used to predict a sentiment score in a document / email.

For that purpose, two inputs are required:
- a list of seed words that caracterize a sentiment 
  Exemple : the seed words ["mad", "furious", "insane"] caracterize the sentiment "dissatisfaction"
- a trained embedding (Melusine **Embedding** class instance) to compute distances between words/tokens

The three steps for sentiment score prediction are the following:
- Instanciate a SentimentDetector object with a list of seed words as argument
- Use the SentimentDetector.fit method (with an embedding object as argument) to compute the lexicons
- Use the SentimentDetector.predict method on a document/email DataFrame to predict the sentiment score

## Minimal working exemple

In [2]:
import pandas as pd
import numpy as np

# NLP tools
from melusine.nlp_tools.embedding import Embedding
from melusine.nlp_tools.tokenizer import Tokenizer

# Models
from melusine.models.modeler_semantic import SemanticDetector

### Load email data

In [3]:
df_emails_clean = pd.read_csv('./data/emails_preprocessed.csv', encoding='utf-8', sep=';')
df_emails_clean = df_emails_clean[['clean_body']]
df_emails_clean = df_emails_clean.astype(str)

### Embedding

In [4]:
# Train an embedding using the text data in the 'clean_body' column
embedding = Embedding(input_column='clean_body', size=300, min_count=2)
embedding.train(df_emails_clean)

In [5]:
# Print a list of words present in the Embedding vocabulary
list(embedding.embedding.vocab.keys())[:3]

['client', 'chez', 'pouvez']

In [6]:
# Test the trained embedding : print most similar words
embedding.embedding.most_similar('client', topn=3)

[('lundi', 0.11570599675178528),
 ('parking', 0.0929856076836586),
 ('comme', 0.0910312682390213)]

### Tokenizer

In [7]:
# Tokenize the text in the clean_body column
tokenizer = Tokenizer (input_column='clean_body', stop_removal=True, n_jobs=20)
df_emails_clean = tokenizer.fit_transform(df_emails_clean)

In [8]:
# Test the tokenizer : print tokens
df_emails_clean['tokens'].head()

0    [client, chez, pouvez, etablir, flag_name_, fi...
1    [informe, nouvelle, immatriculation, enfin, fa...
2    [suite, a, conversation, telephonique, mardi, ...
3    [fais, suite, a, mail, envoye, bulletin, salai...
4    [voici, ci, joint, bulletin, salaire, comme, d...
Name: tokens, dtype: object

### Instanciate and fit the Sentiment Detector

In [9]:
seed_word_list = ['immatriculation']

# Instanciate a SentimentDetector object
semantic_detector = SemanticDetector(base_seed_words=seed_word_list, tokens_column='tokens')

# Fit the SentimentDetector using the trained embedding
semantic_detector.fit(embedding=embedding)

In [10]:
print('List of seed words:')
print(semantic_detector.seed_list)

List of seed words:
['immatriculation']


In [11]:
seed_word = semantic_detector.seed_list[0]
lexicon = semantic_detector.lexicon_dict[seed_word]
sorted_lexicon = dict(sorted(lexicon.items(), key = lambda x: x[0]))

print(f'(Part of) Lexicon associated with the seed word "{seed_word}":')
for word, sentiment_score in list(sorted_lexicon.items())[:10]:
    print('  ' + word + ' : ' + str(sentiment_score))

(Part of) Lexicon associated with the seed word "immatriculation":
  00 : -0.01054505
  1 : -0.028735867
  2 : 0.034027718
  a : -0.05048247
  adresse : -0.12165247
  afin : 0.03517899
  ainsi : -0.05758333
  assurance : 0.061626766
  assurer : 0.06983626
  attached : -0.027480457


### Predict and print the sentiment score

**Warning :** In this exemple, the embedding is trained on a corpus of 40 emails which is WAY too small to yield valuable results

In [12]:
# Choose the name of the column returned (default is "score")
return_column = "semantic_score"

# Predict the sentiment score on each email of the DataFrame
df_emails_clean = semantic_detector.predict(df_emails_clean, return_column=return_column)

# Print emails with the maximum sentiment score
df_emails_clean.sort_values(by=return_column, ascending=False).head()

,clean_body,tokens,semantic_score
17,vous trouverez ci-joint le certificat de cessi...,"[trouverez, ci-joint, certificat, cession, att...",0.058515
6,j emmenage dans un nouveau studio le vendredi ...,"[emmenage, nouveau, studio, vendredi, flag_dat...",0.054315
7,je me permets de venir vers vous car depuis le...,"[permets, venir, vers, car, depuis, debut, ann...",0.049436
15,je vous remercie pour l attestation demandee p...,"[remercie, attestation, demandee, telephone, r...",0.045309
8,voici la copie du virement effectuer a ce jour...,"[voici, copie, virement, effectuer, a, jour, s...",0.042764


## The SentimentDetector class

The SemanticDetector class provides an unsupervised methodology to assign a sentiment score to a corpus of documents/emails. The methodology used to predict a sentiment score using the SemanticDetector is described below:

1. **Define a list of seed words that caracterize a sentiment**
    - Take a list of seed words as input
    - If the `extend_seed_word_list` parameter is set to True: extend the list of seed words with words sharing the same root (dance -> ["dancing", "dancer"])  
    
    
2. **Fit the model (= create a lexicon for each one of the selected seed words)**
    - Create a lexicon for each seed word by computing the cosine similarity between the seed word and all the words in the vocabulary is computed. 
    - To compute cosine similarities, a trained embedding is required.  
    
    
3. **Predict a sentiment score for emails/documents**
    - Filter out the tokens in the document that are not in the vocabulary.
    - For each remaining token, compute its sentiment score using the different lexicons.
    - For each token, aggregate the score accross different seeds
    - For each email, aggregate the score accross different tokens

The arguments of a SemanticDetector object are :
    
- **base_seed_words :** the list of seed words that caracterize a sentiment
- **tokens_column :** name of the column in the input DataFrame that contains tokens
- **extend_seed_word_list :** if True: complement seed words with words sharing the same root (dance -> ["dancing", "dancer"]). Default value False.
- **normalize_scores :** if True: normalize the lexicon scores of eache word. Default value False.
- **aggregation_function_seed_wise :** Function to aggregate the scores associated with a token accross the different seeds. Default function is a max.
- **aggregation_function_email_wise :** Function to aggregate the scores associated with the different tokens in an email. Default function is the 60th percentile.
- **n_jobs :** the number of cores used for computation. Default value, 1.

## Find extra seed words with the `extend_seed_word_list` parameter

The SentimentDetector "extend_seed_word_list" parameter activates the search for extra seed words sharing the same root as the base seed words.  

For example, if "dance" is a base seed word, "extend_seed_word_list" will loop through the words in the embedding vocabulary and find new seed words such as "dancer", "dancing".

In [13]:
# Instanciate a SentimentDetector object
semantic_detector_extended_seed = SemanticDetector(
    base_seed_words=['tel', 'assur'], tokens_column='tokens', extend_seed_word_list=True)

# Fit the SentimentDetector using the trained embedding
semantic_detector_extended_seed.fit(embedding=embedding)

In [14]:
# Print the extended list of seed words
print(semantic_detector_extended_seed.seed_dict)
print(semantic_detector_extended_seed.seed_list)

{'tel': ['telephonique', 'telephone', 'tel'], 'assur': ['assurance', 'assurer']}
['telephonique', 'telephone', 'tel', 'assurance', 'assurer']


## Use a custom function to aggregate lexicon scores

### Parse an email using lexicons
The SentimentDetector creates a Lexicon for each seed word, therefore, when parsing an email, a score is obtained for each token and each lexicon.  

Exemple : 
- Sentence : "Hello, I like ponies"
- Seed word list : ["horse", "animal"]
- Embedding : simulated

Lexicon "horse" :
{
  "apple" : 0.2,
  ...
  "hello" : 0.1,
  ...
  "ponies" : 8.8,
  ...
  "zebra" : 1.2
}  
Lexicon "animal" :
{
  "apple" : 0.1,
  ...
  "hello" : 0.3,
  ...
  "ponies" : 4.8,
  ...
  "zebra" : 6.2
}

Sentence score :
- "horse" score : 0.1 + 0.3 + 0.2 + 8.8
- "animal" score : 0.3 + 0.2 + 0.2 + 4.8


After parsing, N_token * N_seed scores are obtained and an aggregation  function is necessary to obtain a single sentiment_score for the email.

### Aggregate the score obtained for the different tokens

The default aggregation methodology is the following:  
- Seed-wise aggregation : For a token, take the max score accross seed
  - Exemple :  
    ponies_score = 8.8 (lexicon "horse")   
    ponies_score = 4.8 (lexicon "animal")  
    => Score for the "ponies" token = np.max(8.8, 4.8) = 8.8  
  
  
- Email-wise aggregation : Given a list of token scores, take the percentile 60 as the sentiment_score for the email
  - Exemple :  
    token_score_list : [0.3 (hello), 0.3 (i), 0.2 (ponies)]  
    => sentiment_score = np.percentile([0.3, 0.3, 0.2, 8.8], 60) = 0.3  

In [15]:
# Instanciate a SentimentDetector object with custom aggregation function:
# - A mean for the seed-wise aggregation
# - A 95th percentile for the email-wise aggregation

semantic_detector_custom_aggregation = SemanticDetector(
    base_seed_words=['client'], 
    tokens_column='tokens', 
    aggregation_function_seed_wise=np.mean,
    aggregation_function_email_wise=lambda x: np.percentile(x, 95)
)

# Fit the SentimentDetector using the trained embedding
semantic_detector_custom_aggregation.fit(embedding=embedding)

# Predict the sentiment score on each email of the DataFrame
df_emails_clean_custom_aggregation = semantic_detector_custom_aggregation.predict(df_emails_clean)

## Multiprocessing

In [16]:
semantic_detector_multiprocessing = SemanticDetector(
    base_seed_words=['certificat'], 
    tokens_column='tokens', 
    n_jobs = 2
)

# Fit the SentimentDetector using the trained embedding
semantic_detector_multiprocessing.fit(embedding=embedding)

# Predict the sentiment score on each email of the DataFrame
df_emails_multiprocessing = semantic_detector_multiprocessing.predict(df_emails_clean)